# Kya Allen - Stat 414 - HW 7 Raw Analysis

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy as SciPy

In [3]:
TheData = pd.read_csv('HW7data.csv')
TheData.describe()

,x
count,20.000000
mean,80.211928
std,140.490974
min,0.570046
25%,3.052067
50%,8.895037
75%,75.262763
max,512.647885


### If the data is lognormally distributed, then we would expect ln(x) to be normally distributed.
#### we can transform the data, and then do a test for normality using Q-Q plot and regression. 

In [90]:
TransData = pd.DataFrame(np.log(TheData['x']))
TransData.describe()

,x
count,20.000000
mean,2.569273
std,2.155861
min,-0.562039
25%,1.114438
50%,2.184751
75%,4.296495
max,6.239589


### first, for ocmparison we'll perform a standard Shaprio-Wilk test for normality. 

In [13]:
SciPy.stats.shapiro(TransData)
# (Test statistic, P-value)

(0.9385238885879517, 0.22482778131961823)


#### with P > 0.05, this wouldn't be evidence of non-normality of the transormed data (H0: X ~ N), and so we also would not reject that the raw data isn't lognormal.
### But now we'll try a method taught in class, regressing the data on E(x ~ N)

In [84]:
ExpNorm = [-1.86747, -1.4076, -1.13094, -0.92098, -0.74538, -0.59029, -0.44833, -0.39413, -0.18695, -0.06199, 
           0.06199, 0.18695, 0.39413, 0.44833, 0.59029, 0.74538, 0.92098, 1.13094, 1.4076, 1.86747]
fire = np.sort(TransData['x'])
fire
df = pd.DataFrame(columns=['response', 'P'])
df['response'] = fire
df['P'] = ExpNorm

In [87]:

SimpleLinearModel = sm.OLS(df['response'], df['P']).fit()
print(SimpleLinearModel.summary())

                                 OLS Regression Results                                
Dep. Variable:               response   R-squared (uncentered):                   0.383
Model:                            OLS   Adj. R-squared (uncentered):              0.350
Method:                 Least Squares   F-statistic:                              11.77
Date:                Sun, 04 Apr 2021   Prob (F-statistic):                     0.00280
Time:                        10:40:54   Log-Likelihood:                         -47.551
No. Observations:                  20   AIC:                                      97.10
Df Residuals:                      19   BIC:                                      98.10
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

#### Now we'll attempt to detect any nonlinearity in the data by testing for a significant polynomial coefficient

In [88]:
PolynomialModel = sm.OLS.from_formula(formula='response ~ P + np.square(P)', data=df).fit()
print(PolynomialModel.summary())

                            OLS Regression Results                            
Dep. Variable:               response   R-squared:                       0.961
Model:                            OLS   Adj. R-squared:                  0.956
Method:                 Least Squares   F-statistic:                     207.9
Date:                Sun, 04 Apr 2021   Prob (F-statistic):           1.12e-12
Time:                        10:41:12   Log-Likelihood:                -10.857
No. Observations:                  20   AIC:                             27.71
Df Residuals:                      17   BIC:                             30.70
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        2.4289      0.132     18.351   

### Seeing P > 0.05, we fail to reject the Null hypothesis, that the data has a linear relationship, and thus we find that the transformed data is normally distributed, and thus the original data must be lognormally distributed.

## Part B - Summary Statistics

In [92]:
longN = np.exp(TransData.mean() + (TransData.std() / 2))
longN

x    38.367231
dtype: float64

In [93]:
WeirdS1 = np.exp((2 * TransData.mean()) + TransData.std()) 
WeirdS2 = np.exp(TransData.std()) - 1
WeirdS_final = WeirdS1 * WeirdS2
WeirdS_final

x    11239.536698
dtype: float64